In [1]:
#basic package
import numpy as np
import tqdm
import os
import glob
import sys
import math
import time
import re
import pandas as pd
import random
import json
from collections import defaultdict
import shutil
from sklearn.utils import shuffle

#work with image/videos
import cv2
import imageio
import matplotlib.pyplot as plt
import skvideo.io 
from skimage import color, measure
import scipy.misc

In [2]:
#define path
ROOT_DIR = os.path.abspath("../..")
path_all_initial_image = os.path.join(ROOT_DIR,'COR_datasets','images','share_point_ALL_IMAGES')
path_all_new_image1 = os.path.join(ROOT_DIR,'COR_datasets','images','ALL_newly_extracted_images')
path_all_new_image2 = os.path.join(ROOT_DIR,'COR_datasets','images','ALL_PIECEWISE_newly_extracted_images')
path_image4annot = os.path.join(ROOT_DIR,'COR_datasets','images','share_point_IMAGES_4_ANNOTATIONS')

In [3]:
#create directories if not existing to save all images for annotations (one main folder per species_id)
if not os.path.exists(path_image4annot):
    os.makedirs(path_image4annot)
#add annotations folder inside (to put all files from the annotater (stagiaire))

In [4]:
#we will create folder of nbr_images_folder images for each species with target of nbr_images images per species. one folder per species, and
#then subfolder divided into set of nbr_images_folder images.
#For the species with a lot of images, we will choose the most representative set of images by not using the first months
#for example.

#adding annotations folder
for species_path in glob.glob(os.path.join(path_image4annot, '*')):
    if not species_path.endswith('.csv'):
        for folder_path in glob.glob(os.path.join(species_path,'*')):
            if not folder_path.endswith('.json'):
                print(folder_path)
                p = os.path.join(folder_path,'annotations')
                if not os.path.exists(p):
                    #add annotations folder inside (to put all files from the annotater (stagiaire))
                    os.makedirs(p)

# Download images info

In [5]:
#species info
df_species = pd.read_csv(os.path.join(ROOT_DIR, 'COR_documents\\species_family_ID.csv'), sep=';')
dico_ID_species = dict(zip(df_species['Code'], df_species['Nom commun']))
dico_species_ID = {v.upper():k for k,v in dico_ID_species.items()}

In [6]:
#all initial images info
df = pd.read_csv(os.path.join(path_all_initial_image,'image_info.csv'), sep=';')
#is comming from the initial camera (not with same quality as the ne from now)
df['is_cam_01'] = df['camera'].map(lambda x: x.startswith('CAM01'))
df['is_cam_01'].value_counts(normalize=True)*100
df['video_name'] = 'no_name'
print(df.shape)
df.head(3)

(17222, 18)


,annee,barrage,camera,date,espece,espece_id,heure,initial_image_name,jour,local_image_name,local_path,minute,mois,nombre,seconde,taille,is_cam_01,video_name
0,2015,CYPY01,CAM01B01,01.07.15,Brème,BRE,0,01.07.15_002046_BRE_B.jpg,1,CYPY01_CAM01B01_01.07.15_002046_BRE_B.jpg,D:\vm_exchange\COREALIS\COR_datasets\images\sh...,20,7,1.0,46,B,True,no_name
1,2015,CYPY01,CAM01B01,01.07.15,Brème,BRE,1,01.07.15_011807_BRE_B.jpg,1,CYPY01_CAM01B01_01.07.15_011807_BRE_B.jpg,D:\vm_exchange\COREALIS\COR_datasets\images\sh...,18,7,1.0,7,B,True,no_name
2,2015,CYPY01,CAM01B01,01.07.15,Ablette,ABL,5,01.07.15_051025_ABL_A.jpg,1,CYPY01_CAM01B01_01.07.15_051025_ABL_A.jpg,D:\vm_exchange\COREALIS\COR_datasets\images\sh...,10,7,1.0,25,A,True,no_name


In [7]:
#all new images (extracted from video) info ALLINONCE
df_new_image = pd.read_csv(os.path.join(path_all_new_image1,'image_info.csv'), sep=';')
#is comming from the initial camera (not with same quality as the ne from now)
df_new_image['is_cam_01'] = df_new_image['camera'].map(lambda x: x.startswith('CAM01'))
df_new_image['is_cam_01'].value_counts(normalize=True)*100
print(df_new_image.shape)
df_new_image.head(3)

(4108, 18)


,annee,barrage,camera,date,espece,espece_id,heure,initial_image_name,jour,local_image_name,local_path,minute,mois,nombre,seconde,taille,video_name,is_cam_01
0,2016,SEUJ01,CAM03B03,06.09.16,Black bass,BAS,18,06.09.16_180901_BAS_B.und,6,SEUJ01_CAM03B03_06.09.16_180901_BAS_B_537_541_...,D:\vm_exchange\COREALIS\COR_datasets\images\AL...,9,9,1,1,B,537_541_fish-cam03-2016-09-06_18_1.mp4,False
1,2016,SEUJ01,CAM03B03,06.09.16,Black bass,BAS,18,06.09.16_180901_BAS_B.und,6,SEUJ01_CAM03B03_06.09.16_180901_BAS_B_537_541_...,D:\vm_exchange\COREALIS\COR_datasets\images\AL...,9,9,1,1,B,537_541_fish-cam03-2016-09-06_18_6.mp4,False
2,2016,SEUJ01,CAM03B03,20.09.16,Black bass,BAS,7,20.09.16_073316_BAS_B.und,20,SEUJ01_CAM03B03_20.09.16_073316_BAS_B_1992_199...,D:\vm_exchange\COREALIS\COR_datasets\images\AL...,33,9,1,16,B,1992_1996_fish-cam03-2016-09-20_07_1.mp4,False


#all new images (extracted from video) info PIECEWISE
df_new_image_inonce = pd.read_csv(os.path.join(path_all_new_image2,'image_info.csv'), sep=';')
#is comming from the initial camera (not with same quality as the ne from now)
df_new_image_inonce['is_cam_01'] = df_new_image_inonce['camera'].map(lambda x: x.startswith('CAM01'))
df_new_image_inonce['is_cam_01'].value_counts(normalize=True)*100
print(df_new_image_inonce.shape)
df_new_image_inonce.head(3)

# Download images4annotations names (that are in a folder: 0,1,2,3,4...)

In [8]:
li_df = []
for species_path in glob.glob(os.path.join(path_image4annot,'*')):
    species = species_path.split('\\')[-1]
    for species_path_folder in glob.glob(os.path.join(species_path,'*')):
        if species_path_folder.split('\\')[-1].isdigit():
            for img_path in glob.glob(os.path.join(species_path_folder,'*')):
                r = re.compile('[a-z]{3}')
                img_name = img_path.split('\\')[-1]
                if (r.match(img_name.split('.')[-1]) is not None) & (img_name.split('.')[-1] in ['tif', 'jpg', 'png', 'bmp','und']):
                    li_df.append({'img_name':img_name,'img_path':img_path,'espèce':dico_ID_species[species]})

In [9]:
#initial images
li_init = df['local_image_name'].tolist()
li_init = [x[0:-4] for x in li_init]
#all newly created in once
li_new = df_new_image['local_image_name'].tolist()
li_new = [x[0:-4] for x in li_new]
#alltogether
df_annot = pd.DataFrame(li_df)
df_annot['where'] = 'video_piecewise'
df_annot['where'] = df_annot.apply(lambda x: 'initial' if x['img_name'][0:-4] in li_init else x['where'],axis=1)
df_annot['where'] = df_annot.apply(lambda x: 'video_allinonce' if x['img_name'][0:-4] in li_new else x['where'],axis=1)
print(df_annot.shape[0])
df_annot.head(3)

10600


,espèce,img_name,img_path,where
0,Ablette,CYPY01_CAM02B02_01.08.17_071130_ABL_A.jpg,C:\Users\camil\Desktop\animals_code\COREALIS\C...,initial
1,Ablette,CYPY01_CAM02B02_01.10.17_062923_ABL_D.jpg,C:\Users\camil\Desktop\animals_code\COREALIS\C...,initial
2,Ablette,CYPY01_CAM02B02_01.10.17_151059_ABL_C.jpg,C:\Users\camil\Desktop\animals_code\COREALIS\C...,initial


In [10]:
df_annot['where'].value_counts()

initial            6409
video_allinonce    3854
video_piecewise     337
Name: where, dtype: int64

# Summary

In [11]:
#info on repartition of new and initial image per species (for verification)
df1 = df_annot[df_annot['where']=='initial'].groupby(['espèce'])['img_name'].count().reset_index()
df1 = df1.rename(columns={'img_name':'nbr_img_init'})
#
df2 = df_annot[df_annot['where']=='video_allinonce'].groupby(['espèce'])['img_name'].count().reset_index()
df2 = df2.rename(columns={'img_name':'nbr_img_video_allinonce'})
#
df3 = df_annot[df_annot['where']=='video_piecewise'].groupby(['espèce'])['img_name'].count().reset_index()
df3 = df3.rename(columns={'img_name':'nbr_video_piecewise'})
#
df_ = pd.merge(df1,df2, on=['espèce'], how='outer')
df_ = pd.merge(df_,df3, on=['espèce'], how='outer')
df_['nbr_img_init'] = df_['nbr_img_init'].fillna(0)
df_['nbr_img_video_allinonce'] = df_['nbr_img_video_allinonce'].fillna(0)
df_['nbr_video_piecewise'] = df_['nbr_video_piecewise'].fillna(0)
df_['perc_init'] = df_.apply(lambda x: int(round(x['nbr_img_init']/(x['nbr_video_piecewise']+x['nbr_img_video_allinonce']+x['nbr_img_init'])*100,0)), axis=1)
df_['total_nbr_image_4annot'] = df_['nbr_img_init']+df_['nbr_img_video_allinonce']+df_['nbr_video_piecewise']
df_.sort_values('nbr_img_init', ascending=False)

,espèce,nbr_img_init,nbr_img_video_allinonce,nbr_video_piecewise,perc_init,total_nbr_image_4annot
0,Ablette,1200.0,0.0,0.0,100,1200.0
4,Brème,1200.0,0.0,0.0,100,1200.0
7,Gardon,1200.0,0.0,0.0,100,1200.0
1,Barbeau,1185.0,0.0,0.0,100,1185.0
10,Perche,549.0,651.0,0.0,46,1200.0
5,Chevaine,468.0,382.0,0.0,55,850.0
19,Truite lacustre,235.0,751.0,0.0,24,986.0
15,Silure,139.0,653.0,0.0,18,792.0
18,Truite de rivière,134.0,758.0,0.0,15,892.0
14,Rotengle,22.0,215.0,0.0,9,237.0
